In [37]:
import pandas as pd
import numpy as np
import sagemaker
import pickle
from sagemaker.predictor import csv_serializer
from sagemaker import get_execution_role

In [2]:
session=sagemaker.Session()

In [7]:
# Import already trained model using s3 uri of model
container=sagemaker.image_uris.retrieve('xgboost', session.boto_region_name,version='1')
model_data='s3://sagemaker-us-east-1-032934527328/sentimentanalysis/output/xgboost-2021-04-12-16-38-26-824/output/model.tar.gz'
model=sagemaker.model.Model(container,
                            model_data=model_data,
                            role=get_execution_role()
    )

In [10]:
# Batch transform
model_transformer=model.transformer(instance_count=1,instance_type='ml.m5.large')
model_transformer.transform('s3://sagemaker-us-east-1-032934527328/sentimentanalysis/test.csv',
                            content_type='text/csv',split_type='Line')

..........................Arguments: serve
[2021-04-13 16:45:33 +0000] [1] [INFO] Starting gunicorn 19.9.0
[2021-04-13 16:45:33 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
Arguments: serve
[2021-04-13 16:45:33 +0000] [1] [INFO] Starting gunicorn 19.9.0
[2021-04-13 16:45:33 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2021-04-13 16:45:33 +0000] [1] [INFO] Using worker: gevent
[2021-04-13 16:45:33 +0000] [20] [INFO] Booting worker with pid: 20
[2021-04-13 16:45:33 +0000] [21] [INFO] Booting worker with pid: 21
/opt/amazon/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:65: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util (/opt/amazon/lib/python3.7/site-packages/urllib3/

[2021-04-13:16:46:28:INFO] Sniff delimiter as ','
[2021-04-13:16:46:28:INFO] Determined delimiter of CSV input is ','
[2021-04-13:16:46:28:INFO] Sniff delimiter as ','
[2021-04-13:16:46:28:INFO] Determined delimiter of CSV input is ','
[2021-04-13:16:46:31:INFO] Sniff delimiter as ','
[2021-04-13:16:46:31:INFO] Determined delimiter of CSV input is ','
[2021-04-13:16:46:31:INFO] Sniff delimiter as ','
[2021-04-13:16:46:31:INFO] Determined delimiter of CSV input is ','
[2021-04-13:16:46:31:INFO] Sniff delimiter as ','
[2021-04-13:16:46:31:INFO] Determined delimiter of CSV input is ','
[2021-04-13:16:46:31:INFO] Sniff delimiter as ','
[2021-04-13:16:46:31:INFO] Determined delimiter of CSV input is ','
[2021-04-13:16:46:38:INFO] Sniff delimiter as ','
[2021-04-13:16:46:38:INFO] Determined delimiter of CSV input is ','
[2021-04-13:16:46:38:INFO] Sniff delimiter as ','
[2021-04-13:16:46:38:INFO] Determined delimiter of CSV input is ','
[2021-04-13:16:46:38:INFO] Sniff delimiter as ','
[2021-

In [13]:
with open('data-sentiment/data-sentiment-processed.pkl','rb') as f:
    processed_data=pickle.load(f)

In [16]:
!aws s3 cp --recursive s3://sagemaker-us-east-1-032934527328/xgboost-2021-04-13-16-41-24-289/test.csv.out data-sentiment/

In [17]:
y_test= processed_data['test']['labels']
y_pred=np.loadtxt('data-sentiment/test.csv.out',delimiter=',')
y_pred[y_pred>0.5]=1
y_pred[y_pred<=0.5]=0

In [18]:
np.mean(y_pred==y_test)

0.85064

In [19]:
# Endpoint deploy
model.deploy(initial_instance_count=1,instance_type='ml.m5.large')

-------------!

In [41]:
# import endpoint name from s3 or can also be retrieved on  the model.deploy object using model_predictor.endpoint_name
# important to add serializer argument
model_predictor=sagemaker.predictor.Predictor(
    endpoint_name='xgboost-2021-04-13-17-28-42-331',
    sagemaker_session=session,
    serializer=sagemaker.serializers.CSVSerializer()
)

In [42]:
sample=pd.read_csv('data-sentiment/sagemakerready/data/test.csv',header=None,nrows=1)
sample.to_numpy().flatten()

array([0, 0, 0, ..., 0, 0, 0])

In [43]:
model_predictor.predict(sample.to_numpy().flatten())

b'0.45368263125419617'

In [45]:
model_predictor.predict((sample.to_numpy()+1).flatten())

b'0.3295152485370636'

In [47]:
model_predictor.delete_endpoint()